In [6]:
import pandas as pd
import jieba
import pickle

In [2]:
PATH_PN = '../data/taobao-comment/pn.csv'

In [ ]:
neg = pd.read_excel('../data/taobao-comment/neg.xls', header=None, index=None)
pos = pd.read_excel('../data/taobao-comment/pos.xls', header=None, index=None)  # 读取训练语料完毕

neg['text'] = neg[0]
pos['text'] = pos[0]
pos['mark'] = 1
neg['mark'] = 0  # 给训练语料贴上标签

pn = pd.concat([pos, neg], ignore_index=True)  # 合并语料
pn = pn[['text', 'mark']]
permutation = np.random.permutation(pn.shape[0])
pn = pn.iloc[permutation]
print(pn.head(5))
pn.to_csv('../data/taobao-comment/pn.csv', index=False)

In [3]:
pn = pd.read_csv(PATH_PN)

In [4]:
print(pn.head(5))

                                                text  mark
0    用起来还不错，本人还有几张2000-100 1000-50 东券要的加qq 673946022     1
1  2006年12月16号住了一晚，因为看过ctrip的住户点评区，直到比较吵，专门要了一间8楼...     0
2  1.机身采用喷漆，很有质感,手感也很不错的.2.屏幕颜色过渡柔和，显示效果细腻.3.按键面积...     1
3  完美屏，第一次买NOTEBOOK，手神还不错嘛DDR3的条子，蓝牙，WIFI，你想得到的它帮你配齐     1
4             所谓升职，过程中掺杂了太多的勾心斗角，我自己不喜欢这样的风格，纯属个人观点。     0


In [7]:
pn['words'] = pn['text'].apply(lambda x: list(jieba.cut(x)))

Building prefix dict from the default dictionary ...


Loading model from cache /tmp/jieba.cache


Loading model cost 0.794 seconds.


Prefix dict has been built succesfully.


In [13]:
print(pn['words'].head(5))

0    [用, 起来, 还, 不错, ，, 本人, 还有, 几张, 2000, -, 100,  ,...
1    [2006, 年, 12, 月, 16, 号, 住, 了, 一晚, ，, 因为, 看过, c...
2    [1, ., 机身, 采用, 喷漆, ，, 很, 有, 质感, ,, 手感, 也, 很, 不...
3    [完美, 屏, ，, 第一次, 买, NOTEBOOK, ，, 手神, 还, 不错, 嘛, ...
4    [所谓, 升职, ，, 过程, 中, 掺杂, 了, 太, 多, 的, 勾心斗角, ，, 我,...
Name: words, dtype: object


In [14]:
from gensim.models import Word2Vec

Using TensorFlow backend.


In [15]:
import multiprocessing

In [16]:
model_word2vec = Word2Vec(pn['words'], size=64, window=5, min_count=5, workers=multiprocessing.cpu_count())

In [17]:
model_word2vec['我']

array([-0.70475632, -3.13468909,  0.46770158,  1.2176801 , -0.05716988,
       -1.11763108, -1.04645038,  0.18612705,  0.2793124 ,  1.89003778,
       -0.76277429,  0.52214617,  0.19929956, -0.42592835, -0.07852405,
        0.24108826, -0.36993554,  2.08380532,  0.04404699,  0.29671344,
       -1.34178758, -0.7705968 ,  1.45506966, -0.03248395, -0.71018058,
        0.92320287,  0.36943343,  0.79457504,  0.33891708,  1.64407682,
        2.19278812, -1.15802133, -0.62688446,  0.3780694 ,  1.39938617,
       -0.76018089, -1.89643824,  0.45279738,  0.27944747,  0.2496397 ,
        1.61094046,  0.21132395, -1.60298753, -0.82045156,  0.3762545 ,
        3.25001216,  0.28707173,  0.8503629 ,  0.56816649, -1.48735416,
       -0.51098305, -1.30979955,  0.78904366,  1.09420216, -0.12752515,
       -0.53096753, -1.78223431, -1.16513705, -0.31322289, -0.2370282 ,
        0.90583128, -1.16366589, -0.22632426, -1.51436877], dtype=float32)

In [47]:
vocab = dict([(k, v.index+1) for k, v in model_word2vec.wv.vocab.items()])

In [48]:
print(model_word2vec.wv)

In [49]:
import json

In [50]:
print(json.dumps(vocab))

In [51]:
for i in vocab:
    if vocab[i] == 0:
        print(i)

In [52]:
json.dump(vocab, open('../data/taobao-comment/tokenizer', 'w'))

In [53]:
from collections import defaultdict

In [57]:
tokenizer = defaultdict(lambda: 0)

In [60]:
tokenizer['我']

6

In [59]:
tokenizer.update(vocab)

In [63]:
idx = [tokenizer[x] for x in tokenizer]

In [64]:
print(len(idx), len(set(idx)))